In [16]:
import os

In [17]:
runs = {
    'Baseline': [
        ('k00115t', None),
        ('k00126t', 0), # diverged almost at the beginning
        ('k00131t', None),
        ('k00134t', None),
        ],
    '+Aug': [
        ('k00122t', None),
        ('k00127t', 2203), # started diverging later
        ('k00130t', None),
        ('k00136t', None),
        ],
    '+Clear': [
        ('k00123t', None),
        ('k00128t', 2704), # started diverging later
        ('k00129t', None),
        ('k00135t', None),
        ],
    'Ours': [
        ('k00103t', None),
        ('k00124t', None),
        ('k00132t', None),
        ('k00133t', None),
        ]
}
runs

{'Baseline': [('k00115t', None),
  ('k00126t', 0),
  ('k00131t', None),
  ('k00134t', None)],
 '+Aug': [('k00122t', None),
  ('k00127t', 2203),
  ('k00130t', None),
  ('k00136t', None)],
 '+Clear': [('k00123t', None),
  ('k00128t', 2704),
  ('k00129t', None),
  ('k00135t', None)],
 'Ours': [('k00103t', None),
  ('k00124t', None),
  ('k00132t', None),
  ('k00133t', None)]}

In [18]:
rundir = '/mnt/proj2/open-26-1/out/skygan256/'
rundirs = os.listdir(rundir)

In [19]:
def rundir_path_from_short(short_run_name):
    assert short_run_name[0] == 'k' # ran on Karoline
    assert short_run_name[-1] == 't' # translation-invariant
    rundir_matches = list(filter(lambda rd: rd.startswith(short_run_name[1:-1]), rundirs))
    assert len(rundir_matches) == 1
    return os.path.join(rundir, rundir_matches[0])
rundir_path_from_short('k00103t')

'/mnt/proj2/open-26-1/out/skygan256/00103-stylegan3-t-auto_processed_20230405_1727-gpus8-batch32-gamma2'

In [20]:
import re
def short_from_rundir_path(rundir_path):
    #print(rundir_path)
    m = re.match('.*/(\d{5})-stylegan.*', rundir_path)
    assert m is not None
    return 'k'+m[1]+'t'
short_from_rundir_path(rundir_path_from_short('k00103t'))

'k00103t'

In [21]:
def get_snapshot_files(dirname, limit=None):
    #print(f'get_snapshot_files({dirname}, {limit})')
    return sorted(list(filter(
        lambda f:
            f.startswith('network-snapshot-') and
            f.endswith('.pkl') and
            (limit is None or int(f.replace('network-snapshot-', '').replace('.pkl', '')) <= limit),
        os.listdir(dirname)
        )))
#get_snapshot_files(rundir_path_from_short('k00103t'))
get_snapshot_files(rundir_path_from_short('k00103t'), 40)

['network-snapshot-000000.pkl',
 'network-snapshot-000020.pkl',
 'network-snapshot-000040.pkl']

In [22]:
def kimgs_from_snapshot_file(f):
    return int(f[-10:-4])
kimgs_from_snapshot_file(get_snapshot_files(rundir_path_from_short('k00103t'))[-1])

8733

In [23]:
snapshot_lists = {}

for runname, short_run_names_and_limits in runs.items():
    print('Run', runname)
    offset=0
    numbered_snapshots = {}
    for short_run_name, limit in short_run_names_and_limits:
        snapshots = get_snapshot_files(rundir_path_from_short(short_run_name))
        last_snapshot_kimgs = kimgs_from_snapshot_file(snapshots[-1])
        snapshots_limited = get_snapshot_files(rundir_path_from_short(short_run_name), limit)
        last_snapshot_kimgs_limited = kimgs_from_snapshot_file(snapshots_limited[-1])
        print(f'\t{short_run_name}: from {offset} to {offset + last_snapshot_kimgs}' + (f' (using {last_snapshot_kimgs_limited} from this run; total up to {offset + last_snapshot_kimgs_limited})' if limit is not None else ''))
        for snapshot in snapshots_limited:
            k = offset + kimgs_from_snapshot_file(snapshot)
            v = os.path.join(rundir_path_from_short(short_run_name), snapshot)
            if k in numbered_snapshots.keys():
                assert v.endswith('network-snapshot-000000.pkl')
            else:
                numbered_snapshots[k] = v
            #print(f'\t\t{snapshot}: {kimgs_from_snapshot_file(snapshot)}')
        offset += last_snapshot_kimgs_limited
    #print(numbered_snapshots)
    snapshot_lists[runname] = numbered_snapshots

Run Baseline
	k00115t: from 0 to 12059
	k00126t: from 12059 to 14823 (using 0 from this run; total up to 12059)
	k00131t: from 12059 to 24899
	k00134t: from 24899 to 30968
Run +Aug
	k00122t: from 0 to 11818
	k00127t: from 11818 to 14342 (using 2203 from this run; total up to 14021)
	k00130t: from 14021 to 27001
	k00136t: from 27001 to 30506
Run +Clear
	k00123t: from 0 to 10917
	k00128t: from 10917 to 13621 (using 2704 from this run; total up to 13621)
	k00129t: from 13621 to 25900
	k00135t: from 25900 to 30447
Run Ours
	k00103t: from 0 to 8733
	k00124t: from 8733 to 20972
	k00132t: from 20972 to 26540
	k00133t: from 26540 to 30546


In [24]:
import queue

def shuffle_binary_search(lst): # first, last, middle, then "recurse" to (first, middle) and (middle, last)
    assert type(lst) == list
    def mid_iter2(a,b):
        q=queue.Queue()
        q.put((a+1,b))
        yield a
        yield b
        while not q.empty():
            a,b=q.get()
            if a>=b:
                continue
            mid = a+(b-a)//2
            yield mid
            q.put((a,mid))
            q.put((mid+1,b))

    l=len(lst)-1
    #print('indices', list(mid_iter2(0,l)))

    for i in mid_iter2(0,l):
        yield lst[i]

#list(shuffle_binary_search([1,2,3,4,5])) # -> [1, 5, 3, 2, 4]

In [25]:
# shuffled_snapshot_lists['Baseline'] == list of tuples (kimgs, path)
shuffled_snapshot_lists = {runname: list(shuffle_binary_search([(kimgs,fname) for kimgs,fname in data.items()])) for runname, data in snapshot_lists.items()}

# Generate job scripts

In [26]:
def get_command(runname, kimgs, pkl_file):
    d = {
        #name       use_encoder     azimuth_normalization
        'Baseline': [False,         False],
        'Clear':   [True,          False],
        'Aug':     [False,         True],
        'Ours':     [True,          True],
    }
    assert runname in d.keys()

    options = d[runname]

    return f'compute_one_fid {kimgs} {pkl_file} {options[0]} {options[1]}'

In [27]:
def script_head(runname):
    return f'''#!/bin/bash
#PBS -N sFID_{runname}
#PBS -q qgpu
#PBS -l select=1,walltime=3:00:00
#PBS -A OPEN-26-1

#cd /home/martinmcgg/skygan-docker_1024
cd /home/martinmcgg/skygan-docker_1024_aug

. "/mnt/proj2/open-26-1/miniconda3/etc/profile.d/conda.sh"
conda init bash
conda activate stylegan3

export PY=python3.9
export CACHE_DIR=/scratch/project/open-26-1/martinmcgg/skygan-diskcache
export OPENCV_IO_ENABLE_OPENEXR=1
export CUDA_HOME='/usr/local/cuda'

nvidia-smi
set

cd src/stylegan3/

mkdir -p /mnt/proj2/open-26-1/out/skygan256_fid_{runname}

compute_one_fid() (
    KIMG="$1"
    PKL="$2"
    ENC="$3"
    AZI="$4"
    echo $PKL >> {'/home/martinmcgg/'+runname+'.log'}

$PY train.py \\
--cfg=stylegan3-t --gpus=1 \\
--data /mnt/proj2/open-26-1/projects/SkyGAN/clouds_fisheye/auto_processed/auto_processed_20230405_1727.csv \\
--resolution=256 --gamma=2 \\
--batch=32 --batch-gpu=4 --tick=10 --snap=2 \\
--outdir=/mnt/proj2/open-26-1/out/skygan256_fid_{runname} \\
--metrics=fid50k_full \\
--mirror=0 \\
--aug-ada-xflip=0 \\
--aug-ada-rotate90=0 \\
--aug-ada-xint=0 \\
--aug-ada-scale=0 \\
--aug-ada-rotate=1 \\
--aug-ada-aniso=0 \\
--aug-ada-xfrac=0 \\
--normalize-azimuth=$AZI \\
--use-encoder=$ENC \\
--resume="$PKL"
)

'''

for runname, shuffled_list in shuffled_snapshot_lists.items():
    runname = runname.replace('+', '')
    with open('/home/martinmcgg/'+runname+'.sh', 'w') as f_out:
        f_out.write(script_head(runname))
        for kimgs, pkl_file in shuffled_list:
            f_out.write(get_command(runname, kimgs, pkl_file) + '\n')
    #print(shuffled_list)

# Collect FID results

In [28]:
all_short_run_names = [runname for sublist in runs.values() for runname, limit in sublist]
all_short_run_names

['k00115t',
 'k00126t',
 'k00131t',
 'k00134t',
 'k00122t',
 'k00127t',
 'k00130t',
 'k00136t',
 'k00123t',
 'k00128t',
 'k00129t',
 'k00135t',
 'k00103t',
 'k00124t',
 'k00132t',
 'k00133t']

In [29]:
results = {}
fid_file_counter = {run_name:{'fids':0, 'pkls':0} for run_name in all_short_run_names}
for runname, shuffled_list in shuffled_snapshot_lists.items():
    runname = runname.replace('+', '')
    kimgs_list = []
    fid_list = []
    pkl_file_list = []
    for kimgs, pkl_file in shuffled_list:
        short_run_name = short_from_rundir_path(pkl_file)
        #print(short_run_name)
        fid_file_counter[short_run_name]['pkls'] += 1
        fid_fname = pkl_file+'.fid'
        if os.path.exists(fid_fname):
            with open(fid_fname, 'r') as f_in:
                fid_list.append(float(f_in.read()))
                fid_file_counter[short_run_name]['fids'] += 1
            kimgs_list.append(kimgs)
            pkl_file_list.append(pkl_file)

    assert len(kimgs_list) == len(fid_list)
    assert len(kimgs_list) > 0
    with open('/home/martinmcgg/'+runname+'.tsv', 'w') as f_out:
        for kimgs, fid, pkl_file in zip(kimgs_list, fid_list, pkl_file_list):
            f_out.write(f'{kimgs}\t{fid}\t{pkl_file}\n')

for r, counts in fid_file_counter.items():
    if counts['pkls'] > 0:
        print(r, f"{counts['fids']}/{counts['pkls']} ({int(counts['fids']/counts['pkls']*100)} %)")
    else:
        print(r, f"{counts['fids']}/{counts['pkls']}")
    

k00115t 73/603 (12 %)
k00126t 0/0
k00131t 40/641 (6 %)
k00134t 29/303 (9 %)
k00122t 102/591 (17 %)
k00127t 15/110 (13 %)
k00130t 35/648 (5 %)
k00136t 30/175 (17 %)
k00123t 83/546 (15 %)
k00128t 15/135 (11 %)
k00129t 46/613 (7 %)
k00135t 29/227 (12 %)
k00103t 47/437 (10 %)
k00124t 38/611 (6 %)
k00132t 25/278 (8 %)
k00133t 44/200 (22 %)


In [30]:
!pwd

/home/martinmcgg/skygan-docker_1024_aug/src/stylegan3/training
